We are uploading all csv files of the Scarlet & Violet main and special sets, then merging all 13 sets into a master set.

Sets: SV01, SV02, SV03, SV: Scarlet & Violet 151, SV04, SV: Paldean Fates, SV05, SV06, SV: Shrouded Fable, SV07, SV08, SV: Prismatic Evolutions, SV09

In [7]:
import pandas as pd
sv01_df = pd.read_csv('SV01ScarletAndVioletBaseSetProductsAndPrices.csv')
sv02_df = pd.read_csv('SV02PaldeaEvolvedProductsAndPrices.csv')
sv03_df = pd.read_csv('SV03ObsidianFlamesProductsAndPrices.csv')
sv04_df = pd.read_csv('SV04ParadoxRiftProductsAndPrices.csv')
sv05_df = pd.read_csv('SV05TemporalForcesProductsAndPrices.csv')
sv06_df = pd.read_csv('SV06TwilightMasqueradeProductsAndPrices.csv')
sv07_df = pd.read_csv('SV07StellarCrownProductsAndPrices.csv')
sv08_df = pd.read_csv('SV08SurgingSparksProductsAndPrices.csv')
sv09_df = pd.read_csv('SV09JourneyTogetherProductsAndPrices.csv')
sv_151_df = pd.read_csv('SVScarletAndViolet151ProductsAndPrices.csv')
sv_pf_df = pd.read_csv('SVPaldeanFatesProductsAndPrices.csv')
sv_sf_df = pd.read_csv('SVShroudedFableProductsAndPrices.csv')
sv_pe_df = pd.read_csv('SVPrismaticEvolutionsProductsAndPrices.csv')


